# Test 1


In [13]:
#import timm 
import torch
from timm import create_model
from timm.data.transforms_factory import create_transform
from timm.data import resolve_data_config, create_dataset

In [14]:
ds = create_dataset(name='', root='img', transform = create_transform(224))

In [15]:
image, label = ds[0]

In [16]:
image.shape

torch.Size([3, 224, 224])

In [17]:
model = create_model('vit_base_patch16_xp_224.untrained', pretrained=False)
x     = torch.randn(1, 3, 224, 224)
model(x).shape

torch.Size([1, 1000])

In [18]:
#timm.list_models(pretrained=True)

In [19]:
model(ds[0])

AttributeError: 'tuple' object has no attribute 'shape'

# test 2

In [1]:
#import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import timm
from sklearn.model_selection import train_test_split
from torchvision import transforms

In [2]:
import pandas as pd
df = pd.read_csv('img_labels_ALL.csv')

In [3]:
len(df)

2561

In [4]:
# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx, 0]
        img_path = f'img/{img_name}'
        image = Image.open(img_path).convert('RGB')
        label = torch.tensor(self.df.iloc[idx, 2], dtype=torch.long)
        if self.transform:
            image = self.transform(image)
        return image, label

In [5]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [6]:
# Split the dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [8]:
# Create data loaders
train_data = CustomDataset(train_df, transform)
test_data = CustomDataset(test_df, transform)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [9]:
# Initialize the model
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=5)


In [10]:
# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [12]:
# Train the model
for epoch in range(1):  # loop over the dataset multiple times
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.long())  # Ensure labels are long integers
        loss.backward()
        optimizer.step()

print('Finished Training')

In [ ]:
#play sound when done
import os
duration = 1  # seconds
freq = 440  # Hz
os.system('play -nq -t alsa synth {} sine {}'.format(duration, freq))

In [ ]:
#save model after training

from joblib import dump, load

# Save the model to a file
dump(model, 'my_model.joblib')



In [ ]:
# Load the model from the file
loaded_model = load('my_model.joblib')

In [54]:
from PIL import Image

# Load the image
image = Image.open('img/test/10050_1E168_2.png').convert('RGB')

# Apply the same transformations as for the training images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Use your own mean and std if you calculated them
])
image = transform(image)

# Add an extra dimension because the model expects batches
image = image.unsqueeze(0)

# Ensure you are in evaluation mode
model.eval()

# Make a prediction
with torch.no_grad():
    output = model(image)

# Get the predicted class
predicted_class = torch.argmax(output, dim=1)

print(f'Predicted class: {predicted_class.item()}')

Predicted class: 1


# test 3

In [1]:
#imports
import pandas as pd #circular import?
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import timm
from sklearn.model_selection import train_test_split
from torchvision import transforms

from pathlib import Path
from tqdm import tqdm
import os

from IPython.display import Audio

In [2]:
#get data labels
#import pandas as pd
df = pd.read_csv('img_labels_ALL.csv')

In [3]:
# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx, 0]
        img_path = f'img/{img_name}'
        image = Image.open(img_path).convert('RGB')
        label = torch.tensor(self.df.iloc[idx, 2], dtype=torch.long)
        if self.transform:
            image = self.transform(image)
        return image, label
    
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Split the dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Create data loaders
train_data = CustomDataset(train_df, transform)
test_data = CustomDataset(test_df, transform)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [4]:
# Initialize the model
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=5)

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [6]:
# TRAINING LOOP that keeps saving best and last and plays sound when done

# Define paths for saving models
model_dir = Path("models")
best_model_path = model_dir / "best_model.pt"
last_model_path = model_dir / "last_model.pt"

# Initialize best validation loss
best_val_loss = float('inf')

# Wrap both loops with tqdm for progress visualization
for epoch in tqdm(range(1)):  # loop over the dataset multiple times
    for i, data in tqdm(enumerate(train_loader,   0), desc=f"Epoch {epoch +  1}", leave=False):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.long())  # Ensure labels are long integers
        loss.backward()
        optimizer.step()
        break #after the first batch

    # Evaluate the model on the validation set
    #val_loss = evaluate_on_validation_set(model, val_loader)  # Replace with your actual evaluation function

    # Save the current model
    torch.save(model.state_dict(), last_model_path)

    # Check if the current model is better than the best model
    #if val_loss < best_val_loss:
        # best_val_loss = val_loss
        #torch.save(model.state_dict(), best_model_path)

print('Finished Training')
os.system('play -nq -t alsa synth {} sine {}'.format(duration, freq))

100%|██████████| 1/1 [00:56<00:00, 56.74s/it]

Finished Training



sh: play: command not found


32512

## evaluate first sample

In [20]:
# Ensure the model is in evaluation mode
model.eval()

# Get the first sample from the dataset
sample, true_label = test_data[0]

# Pass the sample through the model
with torch.no_grad():
    prediction = model(sample.unsqueeze(0))  # Unsqueeze to add batch dimension

# Convert the prediction to a class label
_, predicted_class = torch.max(prediction, dim=1)

# Print the real and predicted classes
print(f"Real class: {true_label.item()}, Predicted class: {predicted_class.item()}")

# Check if the prediction matches the true label
if predicted_class.item() == true_label.item():
    print("Correct prediction.")
else:
    print("Incorrect prediction.")

# Play a success sound
from mac_alerts import alerts
alerts.play_success()

Real class: 4, Predicted class: 1
Incorrect prediction.


## evaluate test samples

In [22]:
# Ensure the model is in evaluation mode
model.eval()

# Initialize counters for correct and total predictions
correct_predictions =  0
total_predictions =  0

# Iterate over the dataset
for i in tqdm(range(len(test_data)), desc='Evaluating'):
    # Get the sample from the dataset
    sample, true_label = test_data[i]
    
    # Pass the sample through the model
    with torch.no_grad():
        prediction = model(sample.unsqueeze(0))  # Unsqueeze to add batch dimension
    
    # Convert the prediction to a class label
    _, predicted_class = torch.max(prediction, dim=1)
    
    # Check if the prediction matches the true label
    if predicted_class.item() == true_label.item():
        correct_predictions +=  1
    
    total_predictions +=  1

# Calculate the accuracy
accuracy = correct_predictions / total_predictions

# Print the results
print(f"Accuracy: {accuracy:.2f}")
print(f"Total predictions: {total_predictions}")
print(f"Correct predictions: {correct_predictions}")

# Play a success sound
from mac_alerts import alerts
alerts.play_success()

Evaluating:  20%|██        | 105/513 [01:14<04:49,  1.41it/s]


KeyboardInterrupt: 

## parallel evaluations of test set

In [26]:
from sklearn.metrics import classification_report
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def evaluate_single_sample(i):
    # Get the sample from the dataset
    sample, true_label = test_data[i]
    
    # Pass the sample through the model
    with torch.no_grad():
        prediction = model(sample.unsqueeze(0))  # Unsqueeze to add batch dimension
    
    # Convert the prediction to a class label
    _, predicted_class = torch.max(prediction, dim=1)
    
    # Return the true label and the predicted class
    return true_label.item(), predicted_class.item()

# Ensure the model is in evaluation mode
model.eval()

# Shared lists to store true labels and predicted classes
true_labels_list = []
predicted_classes_list = []

# Create a thread pool executor
with ThreadPoolExecutor() as executor:
    # Evaluate all samples in parallel and collect results
    results = list(tqdm(executor.map(evaluate_single_sample, range(len(test_data))), total=len(test_data), desc='Evaluating'))

# Collect the true labels and predicted classes from the results
for true_label, predicted_class in results:
    true_labels_list.append(true_label)
    predicted_classes_list.append(predicted_class)

# Generate the classification report
report = classification_report(true_labels_list, predicted_classes_list)

# Print the classification report
print(report)

# Play a success sound
from mac_alerts import alerts
alerts.play_success()

Evaluating: 100%|██████████| 513/513 [01:23<00:00,  6.14it/s]
/Users/alexandermittet/miniconda3/envs/BA/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/alexandermittet/miniconda3/envs/BA/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/alexandermittet/miniconda3/envs/BA/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` par

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       165
           1       0.17      1.00      0.29        87
           2       0.00      0.00      0.00        84
           3       0.00      0.00      0.00        15
           4       0.00      0.00      0.00       162

    accuracy                           0.17       513
   macro avg       0.03      0.20      0.06       513
weighted avg       0.03      0.17      0.05       513

